**Created by Sanjay Raghuwanshi ** <br/>
[Github](https://github.com/sanjay-raghu) <br/>
[Linkedin](https://www.linkedin.com/in/sanjayiitg/) <br/>
**Sentiment Analysis:** the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.

**What's New** I have added how to deal with data imbalance. Almost all classification task have this problem as number of data of every class if different. For current dataset number of data having positive sentiments is very low relative to data with negative sentiment.

**Solving class imbalaned data**:
- upsampling 
- using class weighted loss function

Using LSTM to classify the movie reviews into positive and negative.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re
from google.colab import drive

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Only keeping the necessary columns.

In [ ]:
drive.mount('/content/gdrive') 
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Lab 5/Sentiment.csv")

# Keeping only the neccessary columns
data = data[['text','sentiment']]

Mounted at /content/gdrive


Data preview

In [ ]:
data.head(20)

text sentiment
0   RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1   RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2   RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3   RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4   RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
5   RT @GregAbbott_TX: @TedCruz: "On my first day ...  Positive
6   RT @warriorwoman91: I liked her and was happy ...  Negative
7   Going on #MSNBC Live with @ThomasARoberts arou...   Neutral
8   Deer in the headlights RT @lizzwinstead: Ben C...  Negative
9   RT @NancyOsborne180: Last night's debate prove...  Negative
10  @JGreenDC @realDonaldTrump In all fairness #Bi...  Negative
11  RT @WayneDupreeShow: Just woke up to tweet thi...  Positive
12  Me reading my family's comments about how grea...  Negative
13  RT @ArcticFox2016: RT @AllenWestRepub "Dear @J...   Neutral
14  RT @pattonoswalt: I loved Scott Walker as Mark...  Positive
15  Hey @ChrisChristie exploiting the tragedy of 9...  Negative
16  RT @CarolCNN: #DonaldTrump under fire for comm...  Negative
17  RT @johncardillo: Guess who had most speaking ...  Negative
18  reason comment is funny 'in case you're ignora...  Negative
19  RT @PamelaGeller: Huckabee: Paying for transge...  Negative

Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [ ]:
data = data[data.sentiment != "Negative"]
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head()

text sentiment
0  rt nancyleegrahn how did everyone feel about t...   Neutral
1  rt scottwalker didnt catch the full gopdebate ...  Positive
2  rt tjmshow no mention of tamir rice and the go...   Neutral
3  rt robgeorge that carly fiorina is trending  h...  Positive
4  rt danscavino gopdebate w realdonaldtrump deli...  Positive

In [ ]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Neutral'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt','')
data.head(20)

4472
6284


text sentiment
0    nancyleegrahn how did everyone feel about the...   Neutral
1    scottwalker didnt catch the full gopdebate la...  Positive
2    tjmshow no mention of tamir rice and the gopd...   Neutral
3    robgeorge that carly fiorina is trending  hou...  Positive
4    danscavino gopdebate w realdonaldtrump delive...  Positive
5    gregabbott_tx tedcruz on my first day i will ...  Positive
7   going on msnbc live with thomasarobes around 2...   Neutral
11   waynedupreeshow just woke up to tweet this ou...  Positive
13   arcticfox2016  allenwestrepub dear jebbush go...   Neutral
14   pattonoswalt i loved scott walker as mark har...  Positive
20   chucknellis cruz has class amp truth that get...  Positive
23   thekevinryder gopdebate vine by dabulldawg88 ...   Neutral
32  the first gopdebate social media reaction and ...   Neutral
33  socialmedia the first gopdebate social media r...   Neutral
39   itsashlyperez listening to fetty wap while pa...   Neutral
42   rowdygentleman if youre planning out a gopdeb...   Neutral
46   palmaceiahome1 the goal of foxnews last night...   Neutral
47   philstockworld from our live chat room gopdeb...   Neutral
48  danscavino gopdebate w realdonaldtrump deliver...  Positive
50   ariannahuff the best and worst from the gopde...   Neutral

In [ ]:
    
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 1946,   66,   71,  412,  500,   23,    2,
         546,  369,   80,   19,   29,  889,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         258,  188, 1197,    2,  448,    1,   19,   29,  261,   45,  168,
           7,  100,  784,   11, 1604, 1198,  890]], dtype=int32)

Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 29, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4302, 29) (4302, 2)
(1076, 29) (1076, 2)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [ ]:
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
34/34 [==============================] - 21s 241ms/step - loss: 0.6444 - accuracy: 0.6313
Epoch 2/15
34/34 [==============================] - 8s 243ms/step - loss: 0.5395 - accuracy: 0.7411
Epoch 3/15
34/34 [==============================] - 8s 246ms/step - loss: 0.4404 - accuracy: 0.8045
Epoch 4/15
34/34 [==============================] - 8s 245ms/step - loss: 0.3916 - accuracy: 0.8259
Epoch 5/15
34/34 [==============================] - 8s 247ms/step - loss: 0.3560 - accuracy: 0.8431
Epoch 6/15
34/34 [==============================] - 8s 240ms/step - loss: 0.3292 - accuracy: 0.8649
Epoch 7/15
34/34 [==============================] - 8s 232ms/step - loss: 0.3105 - accuracy: 0.8680
Epoch 8/15
34/34 [==============================] - 8s 240ms/step - loss: 0.2818 - accuracy: 0.8828
Epoch 9/15
34/34 [==============================] - 8s 243ms/step - loss: 0.2758 - accuracy: 0.8812
Epoch 10/15
34/34 [==============================] - 9s 247ms/step - loss: 0.2525 - accuracy: 0.899

Extracting a validation set, and measuring score and accuracy.

In [ ]:
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)
predict_x=model.predict(X_test) 
Y_pred=np.argmax(predict_x,axis=1)

In [ ]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[464 174]
 [141 297]]
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       638
           1       0.63      0.68      0.65       438

    accuracy                           0.71      1076
   macro avg       0.70      0.70      0.70      1076
weighted avg       0.71      0.71      0.71      1076



Finally measuring the number of correct guesses.  It is clear that finding negative tweets (**class 0**) goes very well (**recall 0.92**) for the Network but deciding whether is positive (**class 1**) is not really (**recall 0.52**). My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

As expected accuracy for positive data is vary low compare to negative, Lets try to solve this problem.

**1. Up-sample Minority Class**

Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal.
There are several heuristics for doing so, but the most common way is to simply resample with replacement.

In [ ]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Neutral']
data_minority = data[data['sentiment'] == 'Positive']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

ZeroDivisionError: ignored

In [ ]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())


In [ ]:
# Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

In [ ]:
# model
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Here we train the Network. We should run much more than 15 epoch, but I would have to wait forever for kaggle, so it is 15 for now.

In [ ]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

In [ ]:
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)
predict_x=model.predict(X_test) 
Y_pred=np.argmax(predict_x,axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

So the class imbalance is reduced significantly recall value for positive tweets (Class 1) improved from 0.54 to 0.77. It is alwayes not possible to reduce it compleatly. 

You may also noticed that the recall value for Negative tweets also decreased from 0.90 to 0.78  but this can be improved using training model to more epocs and tuning the hyperparameters.

In [ ]:
# running model to few more epochs
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)
predict_x=model.predict(X_test) 
Y_pred=np.argmax(predict_x,axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

In [ ]:
twt = ['keep up the good work']
#twt = ['bad job']

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")